In [ ]:
import csv
import numpy as np
import open3d as o3d
from matplotlib import pyplot as plt 
from sklearn.neighbors import KDTree
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import ipywidgets as widgets # for interactive sliders
import pickle
from timeit import timeit
from scipy.spatial import cKDTree

from itertools import product


%load_ext autoreload 
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle
from digiforest_analysis.utils.timing import Timer
timer = Timer()

In [ ]:
DATASET_DIR = "/home/ori/git/realtime-trees/single_trees/clustering_2/"
SLICE_HEIGHT = 0.5
TREE_ID = 63

In [ ]:
file_name = DATASET_DIR + "tree" + str(TREE_ID).zfill(3) + ".pkl"
with open(file_name, 'rb') as file:
    cluster = pickle.load(file)

In [ ]:
slice_height = 2.0
slice_thickness = 2.0
cloud = cluster['cloud'].point.positions.numpy()
slice = cloud[np.logical_and(cloud[:, 2] >= slice_height - slice_thickness / 2, cloud[:, 2] < slice_height + slice_thickness / 2,)][:, :2]
print(len(slice))

# timing_hough = timeit("Circle.from_cloud_hough(slice)", "from __main__ import Circle, slice", number=1000)
# print(f"Hough algo took {timing_hough:.3f} ms")
# timing_new = timeit("new_hough(slice)", "from __main__ import new_hough, slice", number=1000)
# print(f"New algo took {timing_new:.3f} ms")

with timer("OLD"):
    circ_hough = Circle.from_cloud_hough(slice)
with timer("NEW"):
    # circ_new, circles = Circle.from_cloud_ransahc(slice, min_radius=0.05, max_radius=0.5, sampling="weighted", max_points=100)
    circ_new = Circle.from_cloud_ransahc(slice, min_radius=0.05, max_radius=0.5, sampling="weighted", max_circles=500)

print(timer)
plt.scatter(slice[:, 0], slice[:, 1], s=5)
plt.gca().set_aspect('equal', adjustable='box')
# plot circle
plt.gca().add_artist(plt.Circle((circ_hough.x, circ_hough.y), circ_hough.radius, color='r', fill=False))
plt.gca().add_artist(plt.Circle((circ_new.x, circ_new.y), circ_new.radius, color='g', fill=False))
# plt.scatter(circles[:, 0], circles[:, 1], s=5, color='r')

In [ ]:
import colorsys
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
import pickle, os

%load_ext autoreload 
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle
# %matplotlib notebook

path = "/home/ori/git/digiforest_drs/trees/logs/raw"
TREE_ID = 12
tree_path = os.path.join(path, f"tree{TREE_ID:03d}.pkl")
with open(tree_path, 'rb') as file:
    tree : Tree = pickle.load(file)

def align_clouds(clusters):
    trafos_map = [c["info"]["T_sensor2map"] @ c["info"]["axis"]["transform"] for c in clusters]
    clouds_map = [c["cloud"].point.positions.numpy() @ c["info"]["T_sensor2map"][:3, :3].T + c["info"]["T_sensor2map"][:3, 3] for c in clusters]
    mean_position = np.mean([t[:3, 3] for t in trafos_map], axis=0)
    deltas = [t[:3, 3] - mean_position for t in trafos_map]
    deltas = [np.array([d[0], d[1], 0]) for d in deltas]
    return [c - d for c, d in zip(clouds_map, deltas)]



# # make 3d plot of clusters in different colors
# fig = go.Figure()
# for cluster in tree.clusters:
#     cloud = cluster['cloud'].point.positions.numpy()
#     transform = cluster['info']['T_sensor2map']
#     cloud = cloud @ transform[:3, :3].T + transform[:3, 3]
#     fig.add_trace(go.Scatter3d(x=cloud[:, 0], y=cloud[:, 1], z=cloud[:, 2], mode='markers', marker=dict(size=1)))
# fig.show()

# make a 2d plot with a slider that changes the height of the slice
clouds = [
    cluster['cloud'].point.positions.numpy() @ cluster['info']['T_sensor2map'][:3, :3].T + cluster['info']['T_sensor2map'][:3, 3]
    for cluster in tree.clusters
]
clouds_aligned = align_clouds(tree.clusters)
cloud_merged = np.concatenate(clouds, axis=0)
cloud_aligned_merged = np.concatenate(clouds_aligned, axis=0)
# cloud_aligned_merged = tree.points
hues = np.linspace(0, 1, len(clouds) + 1)[:len(clouds)]
colors = [colorsys.hsv_to_rgb(h, 1, 1) for h in hues]

slice_thickness = 0.2
# min and max width and height are 10th and 90 th percentiles
min_width, max_width = np.percentile(cloud_merged[:, 0], [2, 98])
min_height, max_height = np.percentile(cloud_merged[:, 1], [2, 98])
@widgets.interact(height=widgets.FloatSlider(min=cloud_merged[:, 2].min(), max=cloud_merged[:, 2].max(), step=0.3, value=0.0))
def update_slice_height(height):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    for a in ax:
        a.set_aspect('equal', adjustable='box')
        a.set_xlim(min_width, max_width)
        a.set_ylim(min_height, max_height)
    for cloud, color in zip(clouds, colors):
        slice = cloud[np.logical_and(cloud[:, 2] >= height - slice_thickness / 2, cloud[:, 2] < height + slice_thickness / 2,)][:, :2]
        ax[0].scatter(slice[:, 0], slice[:, 1], s=1, color=color)
    for cloud, color in zip(clouds_aligned, colors):
        slice = cloud[np.logical_and(cloud[:, 2] >= height - slice_thickness / 2, cloud[:, 2] < height + slice_thickness / 2,)][:, :2]
        ax[1].scatter(slice[:, 0], slice[:, 1], s=1, color=color)
    slice_merged = cloud_merged[np.logical_and(cloud_merged[:, 2] >= height - slice_thickness / 2, cloud_merged[:, 2] < height + slice_thickness / 2,)][:, :2]
    slice_aligned_merged = cloud_aligned_merged[np.logical_and(cloud_aligned_merged[:, 2] >= height - slice_thickness / 2, cloud_aligned_merged[:, 2] < height + slice_thickness / 2,)][:, :2]
    ax[2].scatter(slice_aligned_merged[:, 0], slice_aligned_merged[:, 1], s=1, color='k')
    # fit hough circle
    center_region = Circle(tree.axis["transform"][:3, 3], tree.axis["radius"])
    min_radius = 0.5 * tree.axis["radius"]
    max_radius = 1.5 * tree.axis["radius"]
    circ_hough = Circle.from_cloud_hough(slice_merged, min_radius=0.05, max_radius=0.5)
    circ_hough_aligned = Circle.from_cloud_hough(slice_aligned_merged, min_radius=0.05, max_radius=0.5)
    circ_ransahc = Circle.from_cloud_ransahc(slice_merged, min_radius=min_radius, max_radius=max_radius, center_region=center_region)
    circ_ransahc_aligned = Circle.from_cloud_ransahc(slice_aligned_merged, min_radius=min_radius, max_radius=max_radius, center_region=center_region)
    ax[0].add_artist(plt.Circle((circ_hough.x, circ_hough.y), circ_hough.radius, color='r', fill=False, linewidth=2))
    ax[0].add_artist(plt.Circle((circ_ransahc.x, circ_ransahc.y), circ_ransahc.radius, color='g', fill=False, linewidth=2))
    ax[2].add_artist(plt.Circle((circ_hough_aligned.x, circ_hough_aligned.y), circ_hough_aligned.radius, color='r', fill=False, linewidth=2))
    ax[2].add_artist(plt.Circle((circ_ransahc_aligned.x, circ_ransahc_aligned.y), circ_ransahc_aligned.radius, color='g', fill=False, linewidth=2))

In [ ]:
import open3d as o3d
import numpy as np
import colorsys
import pickle, os
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle

path = "/home/ori/git/digiforest_drs/trees/logs/raw"
TREE_ID = 50
tree_path = os.path.join(path, f"tree{TREE_ID:03d}.pkl")
with open(tree_path, 'rb') as file:
    tree : Tree = pickle.load(file) 

# clouds = [cluster['cloud'].transform(cluster['info']['T_sensor2map']).point.positions.numpy() for cluster in tree.clusters]
clouds = [cluster['cloud'].point.positions.numpy() @ cluster['info']['T_sensor2map'][:3, :3].T + cluster['info']['T_sensor2map'][:3, 3]for cluster in tree.clusters]
hues = np.linspace(0, 1, len(clouds) + 1) [:len(clouds)]
colors = np.concatenate([np.array([[colorsys.hls_to_rgb(h, 0.6, 1)]]*len(c)) for h, c in zip(hues, clouds)]).squeeze()
# for cluster in tree.clusters:
#     axis_trafo = cluster["info"]['T_sensor2map'] @ cluster["info"]["axis"]["transform"]
#     circle_lower = Circle(axis_trafo[:3, 3], cluster["info"]["axis"]["radius"], axis_trafo[:3, 2])
#     circle_lower = Circle(axis_trafo[:3, 3] + axis_trafo[:3, 2] * cluster)
#     verts, tris = circle_lower.generate_cone_frustum_mesh(circle_upper)
#     verts_vec = o3d.utility.Vector3dVector(verts)
#     tris_vec = o3d.utility.Vector3iVector(
#         np.concatenate((tris, np.flip(tris, axis=1)), axis=0)
#     )
#     terrain_mesh = o3d.geometry.TriangleMesh(verts_vec, tris_vec)
# raise ValueError
cloud_aligned = tree.points
o3d_cloud = o3d.t.geometry.PointCloud(np.vstack(clouds))
o3d_cloud_aligned = o3d.t.geometry.PointCloud(cloud_aligned)
o3d_cloud.point.colors = colors
o3d_cloud_aligned.point.colors = colors

# min_height = 5
# max_height = 6
# mask = np.logical_and(o3d_cloud.point.positions.numpy()[:, 2] > min_height, o3d_cloud.point.positions.numpy()[:, 2] < max_height)
# o3d_cloud = o3d_cloud.select_by_mask(mask)
# mask = np.logical_and(o3d_cloud_aligned.point.positions.numpy()[:, 2] > min_height, o3d_cloud_aligned.point.positions.numpy()[:, 2] < max_height)
# o3d_cloud_aligned = o3d_cloud_aligned.select_by_mask(mask)

aligned_cloud_flag = False

def toggle_point_cloud(vis):
    global aligned_cloud_flag
    current_view = vis.get_view_control().convert_to_pinhole_camera_parameters()
    
    if aligned_cloud_flag:
        vis.clear_geometries()
        vis.add_geometry(o3d_cloud_aligned.to_legacy())
        aligned_cloud_flag = False
        print("Showing Aligned Cloud")
    else:
        vis.clear_geometries()
        vis.add_geometry(o3d_cloud.to_legacy())
        print("Showing Non-Aligned Cloud")
        aligned_cloud_flag = True
    vis.get_view_control().convert_from_pinhole_camera_parameters(current_view, True)

visualizer = o3d.visualization.VisualizerWithKeyCallback()
visualizer.create_window()
visualizer.add_geometry(o3d_cloud.to_legacy())
visualizer.register_key_callback(ord("C"), toggle_point_cloud)
visualizer.run()
visualizer.destroy_window()

In [ ]:
import pickle, os, time
%load_ext autoreload
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle

path = "/home/ori/git/digiforest_drs/trees/logs/raw"
TREE_ID = 63
tree_path = os.path.join(path, f"tree{TREE_ID:03d}.pkl")
with open(tree_path, 'rb') as file:
    tree : Tree = pickle.load(file) 
TIME = time.perf_counter()
print(tree.reconstruct2(max_radius_deviation=100))
print(time.perf_counter() - TIME)   

# # DEBUG inside tree.reconstruct2
# from matplotlib import pyplot as plt
# import matplotlib
# import numpy as np
# import colorsys
# matplotlib.use("TKagg")
# fig, ax = plt.subplots()
# ax.set_xlabel("x")
# ax.set_ylabel("y")
# ax.set_xlim((-0.5, 0.5))
# ax.set_ylim((-0.5, 0.4))
# ax.set_title("Averaging of individual Payload Clouds")
# ax.set_aspect('equal', adjustable='box')
# hues = np.linspace(0, 1, len(ransahc_circles) + 1)[:len(ransahc_circles)]
# colors = [list(colorsys.hls_to_rgb(hue, 0.55, 0.8)) for hue in hues]
# for points, circle, color in zip(debug_slice_points, ransahc_circles, colors):
#     ax.scatter(points[:, 0], points[:, 1], c=[color]*len(points), s=2)
#     ax.add_artist(plt.Circle((circle.x, circle.y), circle.radius, color=color, fill=False))
# ax.add_artist(plt.Circle((circle.x, circle.y), circle.radius, color="g", linestyle='dashed', fill=False, linewidth=5))
# fig.show()

In [ ]:
# load las file
import laspy
import open3d as o3d
translation = np.array([-2707081.0000, -1284724.00, -641.00])
T = np.array([
    [0.557580, -0.830028, -0.012582, -61.373615],
    [0.828709, 0.557450, -0.049911, -9.963353],
    [0.048441, 0.017403, 0.998674, -0.759592],
    [0.000000, 0.000000, 0.000000, 1.000000]])
trees = []
for file in os.listdir("/home/ori/datasets/digiforest_gt_prefor"):
    if file.endswith(".las") and "_translated" in file:
        input_path = os.path.join("/home/ori/datasets/digiforest_gt_prefor", file)
        las_file = laspy.read(input_path)
        points = np.vstack((las_file.x, las_file.y, las_file.z)).T
        trees.append(o3d.t.geometry.PointCloud(points))
        
pcd_full = o3d.io.read_point_cloud("/home/ori/logs/logs_stein_am_rhein/2023-08-08-16-14-48-exp03/combined_cloud_small.pcd")
o3d.visualization.draw_geometries([c.to_legacy() for c in trees] + [pcd_full])


In [ ]:
import numpy as np
import open3d as o3d
import laspy
import csv, os, pickle
from scipy.interpolate import RegularGridInterpolator

from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle
from digiforest_analysis.tasks.terrain_fitting import TerrainFitting
from digiforest_analysis.utils.distances import distance_line_to_line

reco_path = "/home/ori/git/digiforest_drs/trees/logs/raw_sar_exp03/"
gt_path = "/home/ori/datasets/digiforest_gt_prefor_C/"
april_tag_pos_path = "/home/ori/datasets/digiforest_gt_prefor_C/april_tag_locations.csv"
april_tag_measurement_path = "/home/ori/datasets/digiforest_gt_prefor_C/forester_data.csv"
ground_cloud_path = "/home/ori/datasets/digiforest_gt_prefor_C/ground_cloud.las"


# reconstructed trees
reco_trees = []
for file in os.listdir(reco_path):
    if file.endswith(".pkl"):
        with open(os.path.join(reco_path, file), 'rb') as file:
            reco_tree : Tree = pickle.load(file)
            reco_trees.append(reco_tree)

# ground truth trees
id = 0
gt_trees = []
for file in os.listdir(gt_path):
    if file.endswith(".csv") and "_result" in file:
        with open(os.path.join(gt_path, file), 'r') as file:
            csv_reader = csv.DictReader(file)
            circle_stack = []
            for row in csv_reader:
                circle = Circle(
                    np.array([
                        float(row["center_x"]),
                        float(row["center_y"]),
                        float(row["slice_height"])
                    ]), 
                    float(row["radius"]))
                circle_stack.append(circle)
            
            tree = Tree(id)
            tree.reconstructed = True
            tree.circles = circle_stack
            transform = np.eye(4)
            dir_vec = circle_stack[2].center - circle_stack[1].center
            dir_vec /= np.linalg.norm(dir_vec)
            dir_vec_normal = np.array([-dir_vec[1], dir_vec[0], 0])
            dir_vec_normal /= np.linalg.norm(dir_vec_normal)
            transform[:3, 2] = dir_vec
            transform[:3, 0] = dir_vec_normal
            transform[:3, 1] = np.cross(dir_vec, dir_vec_normal)
            transform[:3, 3] = circle_stack[0].center
            tree.clusters = [{
                "info": {
                    "T_sensor2map": np.eye(4),
                    "axis": {
                        "transform": transform,
                        "radius": circle_stack[0].radius
                    }
                }
            }]
            gt_trees.append(tree)
            id += 1

# april tags
with open(april_tag_pos_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    april_tag_positions = {int(row["tag_id"]): np.array([float(row["x"]), float(row["y"]), float(row["z"])]) for row in csv_reader}

# forester DBHs
dbhs = {}
with open(april_tag_measurement_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        if row["Plot"] != "C3":
            continue
        dbhs[int(row["Tree"])] = float(row["DBH [cm]"].replace(",", "."))

# tree matching
tree_pairs = []
for i_gt, gt_tree in enumerate(gt_trees):
    result = {"gt_tree": gt_tree,}
    gt_axis = {"transform": gt_tree.axis["transform"]}
    matches = 0
    for reco_tree in reco_trees:
        reco_axis = {"transform": reco_tree.axis["transform"]}
        dist = distance_line_to_line(gt_axis, reco_axis, clip_heights=[0,10])
        if dist < 0.5:
            result["reco_tree"] = reco_tree
    for k, v in april_tag_positions.items():
        if np.linalg.norm(v[:2] - gt_tree.axis["transform"][:2, 3]) < 0.5:
            result["april_tag"] = k 
            result["forester_dbh"] = dbhs[k]
    if "reco_tree" in result:
        tree_pairs.append(result)

terrain_cloud = laspy.read(ground_cloud_path)
terrain_cloud = np.vstack((terrain_cloud.x, terrain_cloud.y, terrain_cloud.z)).T
terrain_cloud += np.array([-2707081.0000, -1284724.00, -641.00])
terrain_cloud = o3d.t.geometry.PointCloud(terrain_cloud)
terrain = TerrainFitting().process(cloud=terrain_cloud)
terrain_interpolator = RegularGridInterpolator(
    points=(terrain[:, 0, 0], terrain[0, :, 1]),
    values=terrain[:, :, 2],
    method="linear",
    bounds_error=False,
    fill_value=None
)

In [ ]:
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
from digiforest_analysis.utils.meshing import meshgrid_to_mesh
%load_ext autoreload
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle

dbh_height = 1.8
forester_dbhs = []
tls_dbhs = []
reco_dbhs = []
meshes = []
reco_indices = []
for i_tree_pair, tree_pair in enumerate(tqdm(tree_pairs)):
    if "forester_dbh" in tree_pair:
        forester_dbh = tree_pair["forester_dbh"]
        forester_dbhs.append(forester_dbh)
    else: 
        forester_dbhs.append(np.nan)
        
    ground_elevation = terrain_interpolator(tree_pair["gt_tree"].axis["transform"][:2, 3])[0]
    query_height = ground_elevation + dbh_height
    gt_tree = tree_pair["gt_tree"]
    gt_dbh = None
    for i in range(len(gt_tree.circles) - 1):
        circ_height_lower = gt_tree.circles[i].center[2]
        circ_height_upper = gt_tree.circles[i+1].center[2]
        if circ_height_lower < query_height and circ_height_upper > query_height:
            alpha = (query_height - circ_height_lower) / (circ_height_upper - circ_height_lower)
            gt_dbh = (1 - alpha) * gt_tree.circles[i].radius + alpha * gt_tree.circles[i+1].radius
            gt_dbh *= 2
            break
    if gt_dbh is None:
        tls_dbhs.append(np.nan)
        all_fine = False
    else:
        tls_dbhs.append(gt_dbh * 100)
    
    reco_tree = tree_pair["reco_tree"]
    reco_tree.reconstruct2()
    if reco_tree.circles is None:
        reco_dbhs.append(np.nan)
    else:
        reco_dbh = None
        for i in range(len(reco_tree.circles) - 1):
            circ_height_lower = reco_tree.circles[i].center[2]
            circ_height_upper = reco_tree.circles[i+1].center[2]
            if circ_height_lower < query_height and circ_height_upper > query_height:
                alpha = (query_height - circ_height_lower) / (circ_height_upper - circ_height_lower)
                reco_dbh = (1 - alpha) * reco_tree.circles[i].radius + alpha * reco_tree.circles[i+1].radius
                reco_dbh *= 2
                break
        if reco_dbh is None:   
            reco_dbhs.append(np.nan)
        else:
            reco_dbhs.append(reco_dbh * 100)
    
    if gt_dbh is not None and reco_dbh is not None:
        verts, tris = gt_tree.generate_mesh()
        mesh = o3d.geometry.TriangleMesh(
            o3d.utility.Vector3dVector(verts),
            o3d.utility.Vector3iVector(tris)
        )
        mesh.paint_uniform_color([0.2, 0.8, 0.2])
        mesh.compute_vertex_normals()
        meshes.append(mesh)
        verts, tris = reco_tree.generate_mesh()
        mesh = o3d.geometry.TriangleMesh(
            o3d.utility.Vector3dVector(verts),
            o3d.utility.Vector3iVector(tris)
        )
        mesh.paint_uniform_color([0.8, 0.2, 0.2])
        mesh.compute_vertex_normals()
        meshes.append(mesh)
        point_cloud = o3d.t.geometry.PointCloud(reco_tree.points)
        point_cloud.paint_uniform_color([0, 0, 1])
        point_cloud = point_cloud.to_legacy()
        meshes.append(point_cloud)
        reco_indices.append(i_tree_pair)

terrain_verts, terrain_tris = meshgrid_to_mesh(terrain)
mesh = o3d.geometry.TriangleMesh(
    o3d.utility.Vector3dVector(terrain_verts),
    o3d.utility.Vector3iVector(terrain_tris)
)
mesh.compute_vertex_normals()
meshes.append(mesh)

measurements = np.stack([tls_dbhs, forester_dbhs, reco_dbhs], axis=1)
mask = ~np.isnan(measurements).any(axis=1)
measurements = measurements[mask]
sorting = np.argsort(measurements[:, 0])[::-1]
measurements = measurements[sorting]
sorted_and_measured_inds = np.arange(len(reco_dbhs))[mask][sorting]


# compute pearson correlation
from scipy.stats import pearsonr
tls_dbhs = measurements[:, 0]
forester_dbhs = measurements[:, 1]
reco_dbhs = measurements[:, 2]
pearson_forester2tls = pearsonr(tls_dbhs, forester_dbhs)
pearson_reco2tls = pearsonr(tls_dbhs, reco_dbhs)
pearson_reco2forester = pearsonr(forester_dbhs, reco_dbhs)
# compute mean and std error of dbh
diff_forester2tls = forester_dbhs - tls_dbhs
diff_reco2tls = reco_dbhs - tls_dbhs
diff_reco2forester = reco_dbhs - forester_dbhs

print(f"                              mean     median        std    Pearson")
print(f"                              diff       diff       diff      Corr.")
print(f"Forester vs. TLS [cm]:  {np.mean(diff_forester2tls):>10.4f} {np.median(diff_forester2tls):>10.4f} {np.std(diff_forester2tls):>10.4f} {pearson_forester2tls.statistic:>10.4f}")
print(f"Reco vs. TLS [cm]:      {np.mean(diff_reco2tls):>10.4f} {np.median(diff_reco2tls):>10.4f} {np.std(diff_reco2tls):>10.4f} {pearson_reco2tls.statistic:>10.4f}")
print(f"Reco vs. Forester [cm]: {np.mean(diff_reco2forester):>10.4f} {np.median(diff_reco2forester):>10.4f} {np.std(diff_reco2forester):>10.4f} {pearson_reco2forester.statistic:>10.4f}")
print()
print(f"                               MAE     MAPE-l     MAPE-r       RMSE")
print(f"Forester vs. TLS [cm]:  {np.mean(np.abs(diff_forester2tls)):>10.4f} {np.mean(np.abs(diff_forester2tls / forester_dbhs))*100:>8.2f} % {np.mean(np.abs(diff_forester2tls / tls_dbhs))*100:>8.2f} % {np.sqrt(np.mean(diff_forester2tls**2)):>10.4f}")
print(f"Reco vs. TLS [cm]:      {np.mean(np.abs(diff_reco2tls)):>10.4f} {np.mean(np.abs(diff_reco2tls / reco_dbhs))*100:>8.2f} % {np.mean(np.abs(diff_reco2tls / tls_dbhs))*100:>8.2f} % {np.sqrt(np.mean(diff_reco2tls**2)):>10.4f}")
print(f"Reco vs. Forester [cm]: {np.mean(np.abs(diff_reco2forester)):>10.4f} {np.mean(np.abs(diff_reco2forester / reco_dbhs))*100:>8.2f} % {np.mean(np.abs(diff_reco2forester / forester_dbhs))*100:>8.2f} % {np.sqrt(np.mean(diff_reco2forester**2)):>10.4f}")

# compute mean and std axis deviation
errors_dist = []
errors_dist_len = []
errors_radius = []
correlation_x = []
correlation_y = []
correlation_radius = []
centers_reco = []
centers_gt = []
radii_reco = []
radii_gt = []
offset = np.array([0.06354930006341314, 0.11790090669804278, 0])
for i in sorted_and_measured_inds:
    gt_tree : Tree = tree_pairs[i]["gt_tree"]
    reco_tree : Tree = tree_pairs[i]["reco_tree"]
    c_reco = []
    c_gt = []
    r_reco = []
    r_gt = []
    for i_reco in range(len(reco_tree.circles)):
        circ_reco : Circle = reco_tree.circles[i_reco]
        circ_reco.center += offset
        circ_gt = None
        i_gt = 0
        while i_gt < len(gt_tree.circles) - 1:
            if circ_reco.center[2] < gt_tree.circles[i_gt + 1].center[2] and circ_reco.center[2] > gt_tree.circles[i_gt].center[2]:
                alpha = (circ_reco.center[2] - gt_tree.circles[i_gt].center[2]) / \
                (gt_tree.circles[i_gt+1].center[2] - gt_tree.circles[i_gt].center[2])
                center_gt = (1 - alpha) * gt_tree.circles[i_gt].center + alpha * gt_tree.circles[i_gt+1].center
                radius_gt = (1 - alpha) * gt_tree.circles[i_gt].radius + alpha * gt_tree.circles[i_gt+1].radius
                circ_gt = Circle(center_gt, radius_gt)
            i_gt += 1
        if circ_gt is None:
            print(f"Didn't find corresponding circle in ground truth at height {circ_reco.center[2]:.2f}")
            continue
        c_reco.append(circ_reco.center)
        c_gt.append(circ_gt.center)
        r_reco.append(circ_reco.radius)
        r_gt.append(circ_gt.radius)
    errors_dist.append(np.stack([cg - cr for cg, cr  in zip(c_gt, c_reco)]))
    errors_dist_len.append([np.linalg.norm(cg - cr) for cg, cr in zip(c_gt, c_reco)])
    errors_radius.append([er - eg for eg, er in zip(r_gt, r_reco)])
    correlation_x.append(pearsonr([c[0] for c in c_gt], [c[0] for c in c_reco]).statistic)
    correlation_y.append(pearsonr([c[1] for c in c_gt], [c[1] for c in c_reco]).statistic)
    correlation_radius.append(pearsonr(r_gt, r_reco).statistic)
    centers_reco.append(c_reco)
    centers_gt.append(c_gt)
    radii_reco.append(r_reco)
    radii_gt.append(r_gt)
mean_errors_dist = [np.mean(ed) for ed in errors_dist_len]
mean_errors_radius = [np.mean(er) for er in errors_radius]
       
print("")
print(f"                             mean     median        std")
print(f"Distance [cm]:         {np.mean(mean_errors_dist)*100:>10.4f} {np.median(mean_errors_dist)*100:>10.4f} {np.std(mean_errors_dist)*100:>10.4f}")
print(f"Radius [cm]:           {np.mean(mean_errors_radius)*100:>10.4f} {np.median(mean_errors_radius)*100:>10.4f} {np.std(mean_errors_radius)*100:>10.4f}")
print(f"Pearson x:             {np.mean(correlation_x):>10.4f} {np.median(correlation_x):>10.4f} {np.std(correlation_x):>10.4f}")
print(f"Pearson y:             {np.mean(correlation_y):>10.4f} {np.median(correlation_y):>10.4f} {np.std(correlation_y):>10.4f}")
print(f"Pearson radius:        {np.mean(correlation_radius):>10.4f} {np.median(correlation_radius):>10.4f} {np.std(correlation_radius):>10.4f}")
print()
print(f"                              MAE       RMSE      MAPE")
print(f"Distance [cm]:         {np.mean(mean_errors_dist)*100:>10.4f} {np.sqrt(np.mean(np.array(mean_errors_dist)**2))*100:>10.4f}")
print(f"Radius [cm]:           {np.mean(np.abs(np.hstack(errors_radius)))*100:>10.4f} {np.sqrt(np.mean(np.hstack(errors_radius)**2))*100:>10.4f} {((np.abs(np.hstack(errors_radius)) / np.hstack(radii_gt)).mean())*100:>8.2f} %")

fig, ax = plt.subplots()
ax.plot(range(len(forester_dbhs)), forester_dbhs, label="Forester", marker='o', color='r')
ax.plot(range(len(tls_dbhs)), tls_dbhs, label="TLS", marker='o', color='g')
ax.plot(range(len(reco_dbhs)), reco_dbhs, label="Reco", marker='o', color='b')
ax.set_xticks(range(len(sorted_and_measured_inds)), sorted_and_measured_inds)
ax.legend()
fig.show()

# o3d.visualization.draw_geometries(meshes)

In [ ]:
(np.abs(np.hstack(errors_radius)) / np.hstack(radii_gt)).mean()

In [ ]:
import ipywidgets as widgets
%matplotlib inline

@widgets.interact(idx=widgets.IntSlider(min=0, max=len(centers_reco)-1, step=1, value=0))
def update_slice_height(idx):
    fig, ax = plt.subplots(3, 1, figsize=(15, 8))
    print(f"Corr X: {correlation_x[idx]:.3f}, Corr Y: {correlation_y[idx]:.3f}, Corr Radius: {correlation_radius[idx]:.3f}")
    print(f"Mean Error Dist: {mean_errors_dist[idx]*100:.3f} cm, Mean Error Radius: {mean_errors_radius[idx]*100:.3f} cm")
    ax[0].plot(range(len(centers_reco[idx])), [c[0] for c in centers_reco[idx]], label="Reco", marker='o', color='r')
    ax[0].plot(range(len(centers_gt[idx])), [c[0] for c in centers_gt[idx]], label="GT", marker='o', color='g')
    ax[0].set_title("X")
    ax[0].legend()
    ax[1].plot(range(len(centers_reco[idx])), [c[1] for c in centers_reco[idx]], label="Reco", marker='o', color='r')
    ax[1].plot(range(len(centers_gt[idx])), [c[1] for c in centers_gt[idx]], label="GT", marker='o', color='g')
    ax[1].set_title("Y")
    ax[1].legend()
    ax[2].plot(range(len(radii_reco[idx])), radii_reco[idx], label="Reco", marker='o', color='r')
    ax[2].plot(range(len(radii_gt[idx])), radii_gt[idx], label="GT", marker='o', color='g')
    ax[2].set_title("Radius")
    ax[2].legend()
    # fig.show()
 

In [ ]:
# import os
# import laspy
# # trafo correction
# first_trafo = np.array([
#     [0.557580, -0.830028, -0.012582, -61.373615],
#     [0.828709, 0.557450, -0.049911, -9.963353],
#     [0.048441, 0.017403, 0.998674, -0.759592],
#     [0.000000, 0.000000, 0.000000, 1.000000]
# ])
# fine_trafo = np.array([
#     [0.552460, -0.833522, 0.005439, -61.205944],
#     [0.833412, 0.552477, 0.013904, -10.136950],
#     [-0.014595, -0.003148, 0.999888, 1.397392],
#     [0.000000, 0.000000, 0.000000, 1.000000]
# ])
# correction_trafo = fine_trafo @ np.linalg.inv(first_trafo)

# correct trees
# path = "/home/ori/datasets/digiforest_gt_prefor_C"
# for file_name in os.listdir(path):
#     if ".csv" in file_name and "results" in file_name:
#         with open(os.path.join(path, file_name), 'r') as file:
#             csv_reader = csv.DictReader(file)
#             circle_stack = []
#             new_tree = []
#             for row in csv_reader:
#                 circle = Circle(
#                     np.array([
#                         float(row["center_x"]),
#                         float(row["center_y"]),
#                         float(row["slice_height"])
#                     ]), 
#                     float(row["radius"]))
#                 circle.apply_transform(correction_trafo[:3, 3], correction_trafo[:3, :3])
#                 circle_stack.append(circle)
#                 new_tree.append(
#                     {
#                         "slice_height": circle.center[2],
#                         "num_points": row["num_points"],
#                         "center_x": circle.center[0],
#                         "center_y": circle.center[1],
#                         "radius": circle.radius
#                     }
#                 )
#         with open(os.path.join(path, file_name), 'w') as file:
#             print(file_name)
#             csv_writer = csv.DictWriter(file, new_tree[0].keys())
#             csv_writer.writeheader()
#             csv_writer.writerows(new_tree)
        